# Draftnet bot development
This notebook tests the performance of a series of bots from draftsimtools.

### Package importing and parameter setting

In [1]:
# Imports packages 

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data.dataset import Dataset

# Workaround for variable Jupyter directories
import sys
sys.path.append('bots')

import draftsimtools as ds
from draftsimtools import DraftNet

In [2]:
# Sets pytorch device
device = torch.device("cpu") 

# Sets file paths
jsonPath = "../data/AllSets.json"
setRatingPath = "../data/m19_rating.tsv"
landRatingPath = "../data/m19_land_rating.tsv"
draftPath = "../data/subset3000/test.csv"       # Subset drafts
#draftPath = "../../data/m19_2.csv"             # All drafts

### Reads in data
Data folder is downloadable from draft-data-files on slack. 

In [3]:
# Load drafts
raw_drafts = ds.load_drafts(draftPath)

# Load card info from MTGJSON file
with open(jsonPath, 'r',encoding='utf-8') as json_data:
    mtgJSON = json.load(json_data)
jsonSubset = mtgJSON['M19']['cards']
thisSet = {card['name'] : card for card in jsonSubset}

In [4]:
# Another (fancier) way to create a list of names + lots of other useful stuff
nameList = pd.DataFrame.from_dict(thisSet, orient='index') #, columns=['colors','rarity','type','convertedManaCost'])
nameList['Name'] = nameList.index                          # We need names as a column, not an index
nameList['index'] = range(len(nameList))
nameList = nameList.set_index('index')                     # And we need a normal numerical index
nameList[1:5]

# Process names, then handle weird card names (those with commas)
nameList['Name'] = nameList.Name.str.replace(' ', '_')

# This utility method searches for "Name" column in nameList that have commas
nameList, raw_drafts = ds.fix_commas(nameList, raw_drafts) # Returns a tuple, as it updates both

Load sets normally instead.

In [5]:
# Loads sets using draftsimtools utility functions
m19_set = ds.create_set(setRatingPath, landRatingPath)
m19_set = m19_set.sort_values("Name").reset_index(drop=True)

In [6]:
# Processes drafts, deconstructing packs (hands) at every turn of every draft
drafts = ds.process_drafts(raw_drafts)

# Subsets drafts for faster runtimes - for real testing, use all drafts
#subset_drafts = drafts[:20] 
subset_drafts = drafts[500:1000]

Processing draft: 0.


In [7]:
# This converts alternate land names to basic lands in draft data
for iDraft in range(len(subset_drafts)):
    draft = subset_drafts[iDraft]
    for pack in draft:     
        for cardName in pack:
            try:
                pos = nameList[nameList.Name==cardName].index[0]
            except:
                print("---Unrecognized card: ",cardName) # All unrecognized cards here seem to be foil lands
                #  	colors 	rarity 	type 	convertedManaCost 	Name
                nameList = nameList.append({'colors':[],'rarity':'weird','type':'weird',
                                            'convertedManaCost':0,'Name':cardName},ignore_index=True)
print(len(nameList))

---Unrecognized card:  Mountain_3
---Unrecognized card:  Swamp_4
---Unrecognized card:  Island_1
---Unrecognized card:  Mountain_1
---Unrecognized card:  Island_4
---Unrecognized card:  Swamp_3
---Unrecognized card:  Forest_4
---Unrecognized card:  Plains_4
---Unrecognized card:  Island_3
---Unrecognized card:  Mountain_4
---Unrecognized card:  Plains_3
---Unrecognized card:  Forest_1
---Unrecognized card:  Plains_2
---Unrecognized card:  Swamp_2
---Unrecognized card:  Plains_1
---Unrecognized card:  Mountain_2
---Unrecognized card:  Swamp_1
---Unrecognized card:  Island_2
---Unrecognized card:  Forest_3
---Unrecognized card:  Forest_2
320


In [8]:
len(nameList)

320

In [16]:
## Instantiating bots

### Creates static bots

In [18]:
# Instantiates heuristic-based bots
bot1 = ds.RandomBot() 
bot2 = ds.RaredraftBot(nameList) 
bot3 = ds.ClassicBot(nameList, setRatingPath, landRatingPath) 

### Load pre-trained neural network model

In [21]:
# Loads neural net from saved pytorch file
test_net = torch.load("bots/bots_data/draftnet_oct_17_2019_633_cpu.pt")
test_net.eval()

# Loads data (again)
train_data, train_tensor, m19_set, le = ds.load_dataset(setRatingPath, landRatingPath, draftPath)

# Instantiates neural network bot
bot4 = ds.NeuralNetBot(test_net, le)

Processing draft: 0.


### Runs on an example pack

In [11]:
# Instantiates bot tester
tester = ds.BotTester(subset_drafts)

# Create demo collection
demo_collection = tester.drafts[0][0]
demo_pack = tester.drafts[0][1]
demo_x = ds.collection_pack_to_x(demo_collection, demo_pack, le)

# Return the result
result = test_net(demo_x)

# Maps numeric classes to card names and displays result
pack_dict = {str(le.inverse_transform([i])[0]) : float(v.detach().numpy()) for i, v in enumerate(result[0,:]) if v > 0}
display(pack_dict)

{'Anticipate': 12.157964706420898,
 'Cancel': 11.781457901000977,
 'Crash_Through': 10.274678230285645,
 'Daggerback_Basilisk': 13.520963668823242,
 'Dwindle': 14.012452125549316,
 'Epicure_of_Blood': 12.799198150634766,
 'Exclusion_Mage': 16.070850372314453,
 'Hired_Blade': 11.71932315826416,
 'Hostile_Minotaur': 11.677688598632812,
 "Knight's_Pledge": 13.288070678710938,
 'Knightly_Valor': 17.361173629760742,
 'Plains_4': 8.91723918914795,
 'Rise_from_the_Grave': 13.873148918151855,
 'Skeleton_Archer': 12.811566352844238}

## Testing all bots against each other

In [17]:
# Tests all bots in the testing framework
tester.evaluate_bots([bot1, bot2, bot3, bot4], ["RandomBot", "RaredraftBot", "ClassicBot", "NNetBot"])
tester.report_evaluations()

# Writes results to file
tester.write_evaluations()

KeyboardInterrupt: 